# 根据乒乓球点云拟合球体

In [ ]:
import pyransac3d as pyrsc        # RANSAC (Random Sample Consensus) 算法 - 随机抽样一致算法
"""
RANSAC算法假设数据中包含正确数据和异常数据（或称为噪声）
    Inlier: 正确数据记为内点
    Outlier: 异常数据记为外点
"""

In [ ]:
# 乒乓球点云可视化
pcd_load = o3d.io.read_point_cloud(
    "/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/Pingpang/04.pcd")    # 阅读乒乓球点云文件
o3d.visualization.draw_geometries([pcd_load])
points = np.asarray(pcd_load.points)                                       # 将输入的点转化为np array

In [ ]:
# RANSAC 3D拟合球, with center & radius
sph = pyrsc.Sphere()

"""
Sphere函数：
    class Sphere():
        def fit(self, pts, thresh=0.2, maxIteration=1000):
输入参数：
    pts: 三维空间中的点，数据格式为 np.array(N,3)
    thresh: 内点的距离阀值
    maxIteration: RANSAC算法的最大迭代次数
"""

# 输出拟合球体的参数
center, radius, inliers = sph.fit(points, thresh=0.4, maxIteration=100)
print("center: " + str(center))
print("radius: " + str(radius))

"""
输出参数：
    center: 球心的坐标
    radius: 球的半径
    inlier: 内点索引，数据格式为: np.array(1,M)
"""

# 获取内点和外点
inline = pcd_load.select_by_index(inliers).paint_uniform_color([1, 0, 0])                # 选取内点并涂色
outline = pcd_load.select_by_index(inliers, invert=True).paint_uniform_color([0, 1, 0])  # 选取外点并涂色

"""
重要函数：
    select_by_index()              # 一个binary mask从而只输出选定的点
输入参数：
    pts
    Invert = True                  # 选择所有除去已选择点的点
    
重要函数：
    paint_uniform_color([])        # 为点云涂色
"""

# 可视化拟合结果
mesh_circle = o3d.geometry.TriangleMesh.create_sphere(radius=radius)          # 根据半径创建球体
mesh_circle.compute_vertex_normals()                                          # 计算顶点法线
mesh_circle.paint_uniform_color([0.9, 0.1, 0.1])                              # 球体涂色
mesh_circle = mesh_circle.translate((center[0], center[1], center[2]))        # 将球体位移至对应的球心
o3d.visualization.draw_geometries([outline, mesh_circle, inline])             # 将外点+拟合球体+内点可视化

"""
重要代码：
    create_sphere(radius = r)               # 创建一个半径为r的球体
    translate((x,y,z))                      # 将点云平移至(x,y,z)
"""

# 用Ransac拟合分割多个平面

In [ ]:
pcd = o3d.io.read_point_cloud("fil_rect_prism.pcd")
print(pcd)

# 点云降低抽样
print("Downsample the point cloud with a voxel of 0.5") 
downpcd = pcd.voxel_down_sample(voxel_size=0.5)               # 创建一个降低抽样的点云pcd；像素大小为0.5
o3d.visualization.draw_geometries([downpcd])                  # 调用open3D的可视化软件
print(downpcd)

In [ ]:
def identify_planes(pcd, plane_num, dist_threshold = 0.1, ransac_n = 5, num_iterations = 30000):
    '''
    目标：
        识别、分割点云文件中的n个平面，并且计算其平面方程。将每个平面保存为单独的点云并与涂色方便与整体一起展示
    原理：
        使用RANSAC算法随机取出三个点构成平面，并于其余数据进行距离检验。如果误差距离小于阀值（dist_threshold)则加入内点集合。
        当迭代次数耗尽或所有点都被经过计算时停止，并选择内点最多的平面模型。通过反复使用RANSAC进行平面分割并且依次保留平面模型，
        从而获取每个平面的线性方程。
    参数：
        pcd: open3d已阅读的点云
        plane_num: 想要识别的平面次数
        dist_threshold: 定义了一个点到一个平面的最大距离，此距离内的点被认为是内点（距离阀值）
        ransac_n: 定义了使用随机抽样估计一个平面的点的个数（不一定是三个点）
        num_iterations: 定义了随机平面采样和验证的频率（迭代次数）
    返回值：
        print每一个平面的线性方程
        依次visualize每一个平面并且涂色
        返回所有分割平面的点云集合
    '''
        
    plane_list = []                                                                      # 储存分割平面的容器
    i = 1                                                                                # 从1到plane_num的计数
    while i <= plane_num:
        plane_model, inliers = pcd.segment_plane(dist_threshold, ransac_n, num_iterations)   # 通过RANSAC计算平面和内点 
        [a, b, c, d] = plane_model                                                           # 获取平面方程的系数
        print(f"Plane {i} Equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")       # 展示第i个平面的线性方程
    # 注：f-string: substitute the correspond value in the {}
    
        inlier_cloud = pcd.select_by_index(inliers)                                      # 内点点云
        r_color = np.random.uniform(0, 1, (1, 3))                                        # 平面点云随机赋色
        inlier_cloud.paint_uniform_color([r_color[:, 0], r_color[:, 1], r_color[:, 2]])  # 内点随机涂色
        
        outlier_cloud = pcd.select_by_index(inliers, invert=True)                        # 外点点云
        outlier_cloud.paint_uniform_color([0, 1, 0])                                     # 外点涂色
        
        o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud], window_name = 'plane %d' %i)
        answer = input('输入k则保留平面，输入d则删除平面：')
        if answer == 'k':
            plane_list.append(inlier_cloud)                                              # 将内点点云加入plane_list
        elif answer == 'd':
            pass                                                                         # 不保留内点点云

        pcd = outlier_cloud                                                              # 将新的总点云取为外点的点云
        i += 1                                                                           # 迭代至下一个平面

    o3d.visualization.draw_geometries(plane_list, window_name = 'All Segmented Planes')  # 展示所有被分割的平面
    return plane_list

In [ ]:
# 使用identify_planes()函数获得分割平面的方程
plane_list = identify_planes(downpcd, 5)

# 用最小二乘拟合二维曲面

In [ ]:
# 滤波
pcd = o3d.io.read_point_cloud('/Users/chris/Desktop/PC_Modeling_Proj/PointCloud/cur_sur/1/polishROI.ply')
print(pcd)  # 输出点云点的个数
o3d.visualization.draw_geometries([pcd], window_name="原始点云",
                                  width=1024, height=768,
                                  left=50, top=50,
                                  mesh_show_back_face=False)
print("Statistical oulier removal")
cl, ind = pcd.remove_statistical_outlier(nb_neighbors=20,
                                         std_ratio=0.1)
fil_pc = pcd.select_by_index(ind)
o3d.visualization.draw_geometries([fil_pc], window_name="统计滤波",
                                  width=1024, height=768,
                                  left=50, top=50,
                                  mesh_show_back_face=False)

o3d.io.write_point_cloud('fil_curve.pcd', fil_pc)

In [ ]:
import matplotlib.pyplot as plt
from matplotlib import cm

def fun(x):  # 处理符号问题，主要用于表达式输出显示。
    round(x, 2)
    if x >= 0:
        return '+' + str(x)
    else:
        return str(x)


def quadric_surface_fitting(points):
    
    """
    目标：
        根据点云的坐标用最小二乘算法求解出二次曲面方程: z(x,y)=ax^2+bxy+cy^2+dx+ey+f 中的系数
    原理：
        用最小二乘建立目标函数。对各项系数求偏导，并令其等于零，最终得到A和B的线性方程组。
    参数：
        待拟合/点云的坐标
    返回值：
        二次曲面的系数（a,b,c,d,e,f)
    """
    
    X = points[:, 0]
    Y = points[:, 1]
    Z = points[:, 2]
    N = points.shape[0]
    A = np.array([[sum(X ** 4), sum(X ** 3 * Y), sum(X ** 2 * Y ** 2), sum(X ** 3), sum(X ** 2 * Y), sum(X ** 2)],
                  [sum(X ** 3 * Y), sum(X ** 2 * Y ** 2), sum(X * Y ** 3), sum(X ** 2 * Y), sum(X * Y ** 2),
                   sum(X * Y)],
                  [sum(X ** 2 * Y ** 2), sum(X * Y ** 3), sum(Y ** 4), sum(X * Y ** 2), sum(Y ** 3), sum(Y ** 2)],
                  [sum(X ** 3), sum(X ** 2 * Y), sum(X * Y ** 2), sum(X ** 2), sum(X * Y), sum(X)],
                  [sum(X ** 2 * Y), sum(X * Y ** 2), sum(Y ** 3), sum(X * Y), sum(Y ** 2), sum(Y)],
                  [sum(X ** 2), sum(X * Y), sum(Y ** 2), sum(X), sum(Y), N]])
    B = np.array([sum(Z * X ** 2), sum(Z * X * Y), sum(Z * Y ** 2), sum(Z * X), sum(Z * Y), sum(Z)])
    
    # 高斯消元法解线性方程
    res = np.linalg.solve(A, B.T)
    return res


# 主函数
if __name__ == '__main__':
    pcd = o3d.io.read_point_cloud('fil_curve.pcd')
    cloud_points = np.asarray(pcd.points)
    result = quadric_surface_fitting(cloud_points)
    # 输出方程形式
    print("z=%.6s*x^2%.6s*xy%.6s*y^2%.6s*x%.6s*y%.6s" % (fun(result[0]), fun(result[1]), fun(result[2]),
                                                         fun(result[3]), fun(result[4]), fun(result[5])))
    o3d.visualization.draw_geometries([pcd], window_name="可视化输入点云",
                                      width=1024, height=768,
                                      left=50, top=50, mesh_show_back_face=False)

    # 画曲面图和离散点
    fig = plt.figure()                           # 建立一个空间
    ax = plt.axes(projection='3d')               # 建立3D坐标轴
    u = np.linspace(min(X), max(X), 100)         # 创建x和y的line space
    n = np.linspace(min(Y), max(Y), 100)
    x, y = np.meshgrid(u, n)                     # 转化成矩阵
    
    # 给出方程
    z = result[0] * x * x + result[1] * x * y + result[2] * y * y + result[3] * x + result[4] * y + result[5]
    
    # 画出拟合曲面图
    ax.plot_surface(x, y, z, rstride=3, cstride=3, cmap=cm.jet, alpha = 0.7)
    
    '''
    重要函数：
        plot_surface()
    输入参数：
        x,y,z: 2D数组形式的数据值
        rstride: 数组行距（步长大小）
        cstride: 数组列距（步长大小）
        cmap: 曲面块颜色映射
        alpha: 透明度
    '''

    # 画出散点图
    X = cloud_points[:, 0]
    Y = cloud_points[:, 1]
    Z = cloud_points[:, 2]
    ax.scatter(X, Y, Z, c = 'black', s = 1)
    
    '''
    重要函数：
        scatter()
    输入参数：
        x,y,z: 1D(i.e. (n,)格式)的数据值
        s: 点的大小
        c: 颜色
    '''

    plt.show()

In [ ]:
# 前五项误差
print(Z[:5] - (-1.158*X[:5]**2 + 0.063*X[:5]*Y[:5] - 0.371*Y[:5]**2 - 1.612*X[:5] + 0.130*Y[:5] - 0.935))

In [ ]:
# 最大误差
print('最大误差为：' + str(max(Z[:] - (-1.158*X[:]**2 + 0.063*X[:]*Y[:] - 0.371*Y[:]**2 - 1.612*X[:] + 0.130*Y[:] - 0.935))))

# 平均误差
import statistics
print('平均误差为: ' + str(statistics.mean(Z[:] - (-1.158*X[:]**2 + 0.063*X[:]*Y[:] - 0.371*Y[:]**2 - 1.612*X[:] + 0.130*Y[:] - 0.935))))

# 用Open3D同时可视化拟合曲面和点云

In [ ]:
# 生成x,y,z坐标
u = np.linspace(min(X), max(X), 100)         # 创建x和y的line space
n = np.linspace(min(Y), max(Y), 100)
x, y = np.meshgrid(u, n)                     # 转化成矩阵
z = result[0] * x * x + result[1] * x * y + result[2] * y * y + result[3] * x + result[4] * y + result[5]

# 生成.txt点云文件

In [ ]:
print(np.shape(x))

In [ ]:
pts = []                                      # pts = 所有x,y,z坐标的合集
                                              # (i = 0,3,6,9...是x坐标；i = 1,4,7,10...是y坐标；i = 2,5,8,11...是z坐标)
for n in range(len(x)):
    for m in range(len(y)):
        pts.append(x[n,m])
        pts.append(y[n,m])
        pts.append(z[n,m])

with open('curve_pc.txt', 'w') as f:          # 将pts写为一个cloudcompare可以阅读的.txt点云文件
    for i in range(0,len(pts),3):
        f.write(str(pts[i]) + ',' + str(pts[i+1]) + ',' + str(pts[i+2]))
        f.write('\n')

f.close()

# 注：在cc上将.txt文件转换成.pcd的点云

In [ ]:
# 分别创建两个点云geometry
pcd1 = o3d.io.read_point_cloud('curve_pc.pcd')
pcd1.paint_uniform_color([1, 0, 0])   

pcd2 = o3d.io.read_point_cloud('fil_curve.pcd')
pcd2.paint_uniform_color([0, 1, 0]) 

# open3d 可视化拟合曲面和点云
o3d.visualization.draw_geometries([pcd1,pcd2])

# 拟合长方体并且三维重建
### 即：在原本离散的点云中构造连续表面

In [ ]:
import open3d as o3d       # Open 3D 库：用于支持各种三维数据的操作
import numpy as np
import os                  # Operating System (OS) 模块：方便与操作系统进行交互 + 使代码更具有移植性
import sys                 # System (sys) 模块：访问 Python 解释器自身使用和维护的变量

In [ ]:
pcd = o3d.io.read_point_cloud("block1.pcd")
print(pcd)

# 点云降低抽样
print("Downsample the point cloud with a voxel of 0.3") 
downpcd = pcd.voxel_down_sample(voxel_size=0.3)               # 创建一个降低抽样的点云pcd；像素大小为0.3
o3d.visualization.draw_geometries([downpcd])                  # 调用open3D的可视化软件
print(downpcd)

In [ ]:
def identify_planes(pcd, plane_num, dist_threshold = 0.1, ransac_n = 5, num_iterations = 30000):
    '''
    目标：
        识别、分割点云文件中的n个平面，并且计算其平面方程。将每个平面保存为单独的点云并与涂色方便与整体一起展示
    原理：
        使用RANSAC算法随机取出三个点构成平面，并于其余数据进行距离检验。如果误差距离小于阀值（dist_threshold)则加入内点集合。
        当迭代次数耗尽或所有点都被经过计算时停止，并选择内点最多的平面模型。通过反复使用RANSAC进行平面分割并且依次保留平面模型，
        从而获取每个平面的线性方程。
    参数：
        pcd: open3d已阅读的点云
        plane_num: 想要识别的平面次数
        dist_threshold: 定义了一个点到一个平面的最大距离，此距离内的点被认为是内点（距离阀值）
        ransac_n: 定义了使用随机抽样估计一个平面的点的个数（不一定是三个点）
        num_iterations: 定义了随机平面采样和验证的频率（迭代次数）
    返回值：
        print每一个平面的线性方程
        依次visualize每一个平面并且涂色
        返回所有分割平面的点云集合
    '''
        
    plane_list = []                                                                      # 储存分割平面的容器
    i = 1                                                                                # 从1到plane_num的计数
    while i <= plane_num:
        plane_model, inliers = pcd.segment_plane(dist_threshold, ransac_n, num_iterations)   # 通过RANSAC计算平面和内点 
        [a, b, c, d] = plane_model                                                           # 获取平面方程的系数
        print(f"Plane {i} Equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")       # 展示第i个平面的线性方程
    # 注：f-string: substitute the correspond value in the {}
    
        inlier_cloud = pcd.select_by_index(inliers)                                      # 内点点云
        r_color = np.random.uniform(0, 1, (1, 3))                                        # 平面点云随机赋色
        inlier_cloud.paint_uniform_color([r_color[:, 0], r_color[:, 1], r_color[:, 2]])  # 内点随机涂色
        
        outlier_cloud = pcd.select_by_index(inliers, invert=True)                        # 外点点云
        outlier_cloud.paint_uniform_color([0, 1, 0])                                     # 外点涂色
        
        o3d.visualization.draw_geometries([inlier_cloud, outlier_cloud], window_name = 'plane %d' %i)
        answer = input('输入k则保留平面，输入d则删除平面：')
        if answer == 'k':
            plane_list.append(inlier_cloud)                                              # 将内点点云加入plane_list
        elif answer == 'd':
            pass                                                                         # 不保留内点点云

        pcd = outlier_cloud                                                              # 将新的总点云取为外点的点云
        i += 1                                                                           # 迭代至下一个平面

    o3d.visualization.draw_geometries(plane_list, window_name = 'All Segmented Planes')  # 展示所有被分割的平面
    return plane_list

In [ ]:
# 使用identify_planes()函数获得分割平面的方程
plane_list = identify_planes(downpcd, 6, dist_threshold = 0.1, ransac_n = 5, num_iterations = 30000)

In [ ]:
# 将每个平面单独保存为一个点云
pcd_pur = plane_list[0]
pcd_gre = plane_list[1]
pcd_red = plane_list[2]

In [ ]:
# 对每个平面滤波
print("Statistical oulier removal")
cl, ind = pcd_red.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.5)
fil_red = pcd_red.select_by_index(ind)

cl, ind = pcd_pur.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.5)
fil_pur = pcd_pur.select_by_index(ind)

cl, ind = pcd_gre.remove_statistical_outlier(nb_neighbors=20, std_ratio=0.5)
fil_gre = pcd_gre.select_by_index(ind)
o3d.visualization.draw_geometries([fil_red, fil_pur,fil_gre], window_name="统计滤波",
                                  width=1024, height=768,
                                  left=50, top=50,
                                  mesh_show_back_face=False)

In [ ]:
# 可视化分割好的平面与坐标系
mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size = 20, 
                                                         origin = [5.62,0.13105,26.243]) # 红色x轴，绿色y轴，蓝色z轴
o3d.visualization.draw_geometries([fil_pur, fil_gre, fil_red, mesh])

In [ ]:
# 将每个点云中的点记录
pts_pur = np.asarray(fil_pur.points)
pts_gre = np.asarray(fil_gre.points)
pts_red = np.asarray(fil_red.points)

pts_pur_x = []
pts_pur_y = []
pts_pur_z = []
for i in range(len(pts_pur)):
    pts_pur_x.append(pts_pur[i,0])
    pts_pur_y.append(pts_pur[i,1])
    pts_pur_z.append(pts_pur[i,2])
    
pts_red_x = []
pts_red_y = []
pts_red_z = []
for i in range(len(pts_red)):
    pts_red_x.append(pts_red[i,0])
    pts_red_y.append(pts_red[i,1])
    pts_red_z.append(pts_red[i,2])

In [ ]:
# 计算紫色x,y,z的最大值、最小值
print('x_max is: ', max(pts_pur_x), ' with index ', pts_pur_x.index(max(pts_pur_x)))
print('x_min is: ', min(pts_pur_x), ' with index ', pts_pur_x.index(min(pts_pur_x)))

print('y_max is: ', max(pts_pur_y), ' with index ', pts_pur_y.index(max(pts_pur_y)))
print('y_min is: ', min(pts_pur_y), ' with index ', pts_pur_y.index(min(pts_pur_y)))

print('z_max is: ', max(pts_pur_z), ' with index ', pts_pur_z.index(max(pts_pur_z)))
print('z_min is: ', min(pts_pur_z), ' with index ', pts_pur_z.index(min(pts_pur_z)))

In [ ]:
# 算出紫色平面四个角的坐标
print('绿和紫平面的角：', pts_pur[2914,:])      # i.e. x_max
print('只有紫色平面的角：', pts_pur[52,:])      # i.e. y_min
print('紫色和红色的角：', pts_pur[19371,:])     # i.e. x_min
print('三个平面的交点：', pts_pur[10754,:])       # i.e. y_max

In [ ]:
# 计算红色x,y,z的最大值、最小值
print('x_max is: ', max(pts_red_x), ' with index ', pts_red_x.index(max(pts_red_x)))
print('x_min is: ', min(pts_red_x), ' with index ', pts_red_x.index(min(pts_red_x)))

print('y_max is: ', max(pts_red_y), ' with index ', pts_red_y.index(max(pts_red_y)))
print('y_min is: ', min(pts_red_y), ' with index ', pts_red_y.index(min(pts_red_y)))

print('z_max is: ', max(pts_red_z), ' with index ', pts_red_z.index(max(pts_red_z)))
print('z_min is: ', min(pts_red_z), ' with index ', pts_red_z.index(min(pts_red_z)))

In [ ]:
# 算出红和绿平面的角
print('红和绿平面的角：', pts_red[4264,:]) # y_max

In [ ]:
import copy
gre2 = copy.deepcopy(fil_gre).translate([(pts_pur[19371,0] - pts_pur[10754,0]),
                                         (pts_pur[19371,1] - pts_pur[10754,1]),
                                         (pts_pur[19371,2] - pts_pur[10754,2])])
r_color = np.random.uniform(0, 1, (1, 3))                                        
gre2.paint_uniform_color([r_color[:, 0], r_color[:, 1], r_color[:, 2]])

red2 = copy.deepcopy(fil_red).translate([(pts_pur[2914,0] - pts_pur[10754,0]),
                                         (pts_pur[2914,1] - pts_pur[10754,1]),
                                         (pts_pur[2914,2] - pts_pur[10754,2])])
r_color = np.random.uniform(0, 1, (1, 3))                                        
red2.paint_uniform_color([r_color[:, 0], r_color[:, 1], r_color[:, 2]])

pur2 = copy.deepcopy(fil_pur).translate([(pts_red[4264,0] - pts_pur[10754,0]),
                                         (pts_red[4264,1] - pts_pur[10754,1]),
                                         (pts_red[4264,2] - pts_pur[10754,2])])
r_color = np.random.uniform(0, 1, (1, 3))                                        
pur2.paint_uniform_color([r_color[:, 0], r_color[:, 1], r_color[:, 2]])

o3d.visualization.draw_geometries([fil_gre, fil_red, fil_pur, gre2,red2,pur2,mesh])